In [1]:
import pandas as pd
import datetime

import sys
sys.path.append('..')
from utils.utils_iFEMG_feature import readLabel2df, df_save_csv
from utils.utils_signal_preprocess import df_time2stamp, addSuffix

# muscle name VS ch nums
- ch1: RF, Rectus femoris, 股直肌
- ch2: LF, Lateral femoris, 股外侧肌
- ch3: SarT, sartorius, 缝匠肌
- ch4: BF, Biceps femoris, 股二头肌
- ch5: SemiT, semitendinosus, 半腱肌
- ch6: VM, Vastus medialis, 股内侧肌

In [2]:
# 文件名及其对应的key
file_name_dict = {'aqian0':['affected', 'qian', '0'],
                  'aqian1_5':['affected', 'qian', '1.5'],
                  'aqian3':['affected', 'qian', '3'],
                  'ahou0':['affected', 'hou', '0'],
                  'ahou1_5':['affected', 'hou', '1.5'],
                  'ahou3':['affected', 'hou', '3'],
                  'hqian0':['healthy', 'qian', '0'],
                  'hqian1_5':['healthy', 'qian', '1.5'],
                  'hqian3':['healthy', 'qian', '3'],
                  'hhou0':['healthy', 'hou', '0'],
                  'hhou1_5':['healthy', 'hou', '1.5'],
                  'hhou3':['healthy', 'hou', '3']
                  }
# 列名与肌肉名称，仅在传感器通道与上述肌肉位置不对应时需要更改
b1_col_names_lst = ['time', 'FMG_RF', 'FMG_LF', 'FMG_SarT', 'FMG_BF', 'sEMG_RF', 'sEMG_LF', 'sEMG_SarT', 'sEMG_BF']
b2_col_names_lst = ['time', 'FMG_SemiT', 'FMG_VM', 'sEMG_SemiT', 'sEMG_VM']

In [3]:
# 读取数据
# lab path
# folder_path = r"E:\fuckinggggg\paper_iFEMG_4th\data"
label_df = readLabel2df(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b1" + '\\' + 'aqian0.txt')
b1_df = pd.read_table(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b1" + '\\' + 'aqian0.db', header=None, sep=';')
b2_df = pd.read_table(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b2" + '\\' + 'aqian0.db', header=None, sep=';')
# desk path
# b1_df = pd.read_table(r"D:\data\301\241114s1\b1\aqian0.db", sep=';', header=None)
# b2_df = pd.read_table(r"D:\data\301\241114s1\b2\aqian0.db", sep=';', header=None)
# label_lst = read_label(r"D:\data\301\241114s1\b1\aqian0.txt")

# 更改数据列名
b1_df = b1_df.drop(columns=9)
b2_df = b2_df.drop(columns=[3,4,7,8,9])
# s1通道数与其他不同，需要单独修改，其余subjects可以用bx_col_names_lst
b1_df.columns = ['b1_time', 'FMG_RF', 'FMG_SarT', 'FMG_LF', 'FMG_SemiT', 'sEMG_RF', 'sEMG_SarT','sEMG_LF', 'sEMG_SemiT']
b2_df.columns = ['b2_time', 'FMG_BF', 'FMG_VM', 'sEMG_BF', 'sEMG_VM']

In [4]:
# 以下为封装函数需要实现，对每个文件进行操作，获得merged df，添加label标签

# 
b1_df = addSuffix(b1_df, 'b1_time')
b2_df = addSuffix(b2_df, 'b2_time')
data_df = pd.merge(b1_df, b2_df, how='outer', on='suffix')
data_df['time'] = data_df['b1_time'].combine_first(data_df['b2_time'])
data_df

,b1_time,FMG_RF,FMG_SarT,FMG_LF,FMG_SemiT,sEMG_RF,sEMG_SarT,sEMG_LF,sEMG_SemiT,suffix,b2_time,FMG_BF,FMG_VM,sEMG_BF,sEMG_VM,time
0,2024-11-14 15:32:46.708,2991.0,0.0,88.0,0.0,1508.0,289.0,-891.0,721.0,2024-11-14 15:32:46.700_1,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.708
1,2024-11-14 15:32:46.708,2991.0,1469.0,88.0,3238.0,1237.0,283.0,-664.0,1091.0,2024-11-14 15:32:46.700_2,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.708
2,2024-11-14 15:32:46.709,2991.0,1469.0,88.0,3238.0,1398.0,284.0,-637.0,898.0,2024-11-14 15:32:46.700_3,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.709
3,2024-11-14 15:32:46.709,2991.0,1469.0,88.0,3230.0,957.0,285.0,-730.0,635.0,2024-11-14 15:32:46.700_4,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.709
4,2024-11-14 15:32:46.713,2991.0,1469.0,88.0,3230.0,945.0,284.0,-333.0,827.0,2024-11-14 15:32:46.700_5,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239868,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.500_174,2024-11-14 15:35:08.597,3203.0,3823.0,157.0,148.0,2024-11-14 15:35:08.597
239869,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.500_175,2024-11-14 15:35:08.597,3203.0,3814.0,-189.0,61.0,2024-11-14 15:35:08.597
239870,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.500_176,2024-11-14 15:35:08.597,3203.0,3814.0,-65.0,516.0,2024-11-14 15:35:08.597
239871,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.500_177,2024-11-14 15:35:08.597,3203.0,3814.0,139.0,981.0,2024-11-14 15:35:08.597


In [5]:
df_save_csv(data_df, r'E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\test.csv')

File E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\test.csv saved!


In [6]:
data_df = data_df.sort_values('time')
merged_df = pd.merge_asof(data_df, label_df, on='time', direction='backward')
merged_df

,b1_time,FMG_RF,FMG_SarT,FMG_LF,FMG_SemiT,sEMG_RF,sEMG_SarT,sEMG_LF,sEMG_SemiT,suffix,b2_time,FMG_BF,FMG_VM,sEMG_BF,sEMG_VM,time,label
0,2024-11-14 15:32:46.708,2991.0,0.0,88.0,0.0,1508.0,289.0,-891.0,721.0,2024-11-14 15:32:46.700_1,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.708,NaN
1,2024-11-14 15:32:46.708,2991.0,1469.0,88.0,3238.0,1237.0,283.0,-664.0,1091.0,2024-11-14 15:32:46.700_2,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.708,NaN
2,2024-11-14 15:32:46.709,2991.0,1469.0,88.0,3238.0,1398.0,284.0,-637.0,898.0,2024-11-14 15:32:46.700_3,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.709,NaN
3,2024-11-14 15:32:46.709,2991.0,1469.0,88.0,3230.0,957.0,285.0,-730.0,635.0,2024-11-14 15:32:46.700_4,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.709,NaN
4,2024-11-14 15:32:46.713,2991.0,1469.0,88.0,3230.0,945.0,284.0,-333.0,827.0,2024-11-14 15:32:46.700_5,NaT,NaN,NaN,NaN,NaN,2024-11-14 15:32:46.713,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239868,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.500_176,2024-11-14 15:35:08.597,3203.0,3814.0,-65.0,516.0,2024-11-14 15:35:08.597,舒张
239869,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.500_177,2024-11-14 15:35:08.597,3203.0,3814.0,139.0,981.0,2024-11-14 15:35:08.597,舒张
239870,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.500_174,2024-11-14 15:35:08.597,3203.0,3823.0,157.0,148.0,2024-11-14 15:35:08.597,舒张
239871,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.500_175,2024-11-14 15:35:08.597,3203.0,3814.0,-189.0,61.0,2024-11-14 15:35:08.597,舒张


In [8]:
merged_df.to_csv(r'E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\test2.csv', index = False, date_format="%Y-%m-%d %H:%M:%S,%f", encoding='utf-8-sig')

In [19]:
merged_df['label'][342528]

'舒张'

In [ ]:
# 给每行数据添加label
label_col_lst = []
db_index = 0
label_index = 0

while db_index < len(merged_df):
    # 获得时间戳
    db_time = merged_df['b1_time'][db_index] if merged_df['b1_time'][db_index] is not pd.NaT else merged_df['b2_time'][db_index]
    db_stamp = datetime.datetime.strptime(db_time, "%Y-%m-%d %H:%M:%S,%f").timestamp()
    if label_t_stamp[label_index] <= db_stamp < label_t_stamp[label_index]:
        label_col_lst.append(label_lst[label_index][2])




In [36]:
df_save_csv(merged_df, r"D:\data\301\241114s1\test.csv")

File D:\data\301\241114s1\test.csv saved!


In [ ]:
def alienMerge(b1_file: str, b2_file: str, label_file: str)->pd.DataFrame:
    '''
    将单次测试的数据通过时间戳对应，生成dataframe
    '''
    # 读取文件
    try:
        label_data = read_label(label_file)
        b1_data = pd.read_table(b1_file, header=None, sep=';')
        b2_data = pd.read_table(b2_file, header=None, sep=';')
    except FileNotFoundError:
        print(f"file {b1_file} not found")
        return 0
    # 转换时间戳
    
    return